In [15]:
# Open file and parse contents
file = open('day12_inputs.txt')
content = [line.strip() for line in file]

content = [(c[0], int(c[1:])) for c in content]
content[:10]

[('R', 90),
 ('S', 1),
 ('R', 90),
 ('W', 2),
 ('S', 3),
 ('L', 270),
 ('L', 90),
 ('S', 2),
 ('F', 2),
 ('L', 90)]

### Part 1

The inputs are a list of instructions on how to move a ship.

Each instruction contains an action, followed by a value. The actions are:
- N means to move north by the given value.
- S means to move south by the given value.
- E means to move east by the given value.
- W means to move west by the given value.
- L means to turn left the given number of degrees.
- R means to turn right the given number of degrees.
- F means to move forward by the given value in the direction the ship is currently facing.

Follow the instructions, then return the Manhattan distance (absolute value of E/W co-ordinate + absolute value of N/S co-ordinate) relative to it's start point.

In [20]:
dict_directions = {0:'N',
                   90:'E',
                   180:'S',
                   270:'W'}

class Ship:
    
    def __init__(self):
        self.y_coord = 0 # North / South co-ordinate
        self.x_coord = 0 # East / West co-ordinate
        self.degrees = 90
        self.direction = dict_directions[self.degrees]
    
    
    def rotate(self, rotation_direction, degrees):
        
        # Check validity
        if degrees not in [90, 180, 270]:
            raise ValueError('rotation degrees not multiple of 90')
        
        # Get rotation degrees
        clockwise_degrees = -degrees if rotation_direction == 'L' else degrees
        
        # Rotate
        new_degrees = self.degrees + clockwise_degrees
        if new_degrees >= 360:
            new_degrees -= 360
        if new_degrees < 0:
            new_degrees += 360
        
        # Reassign
        self.degrees = new_degrees
        self.direction = dict_directions[self.degrees]
    
    
    def move(self, movement_direction, spaces):
        
        # Move N/S/E/W if directly specified, or infer direction if F specified
        compass_direction = self.direction if movement_direction == 'F' else movement_direction
        
        if compass_direction not in ['N','E','S','W']:
            raise ValueError('Invalid compass_direction')
        elif compass_direction == 'N':
            self.y_coord += spaces
        elif compass_direction == 'E':
            self.x_coord += spaces
        elif compass_direction == 'S':
            self.y_coord -= spaces
        else:
            self.x_coord -= spaces
        
    def get_manhattan_distance(self):
        
        mh_dist = abs(self.x_coord) + abs(self.y_coord)
        print('Direction {}, x coord {}, y coord {}, MH distance {}'.format(self.direction,
                                                                         self.x_coord,
                                                                         self.y_coord,
                                                                         mh_dist))

In [21]:
MyShip = Ship()

for action, num in content:
    
    if action in ['R','L']:
        MyShip.rotate(action,num)
    else:
        MyShip.move(action,num)

MyShip.get_manhattan_distance()

Direction W, x coord 700, y coord -1256, MH distance 1956


### Part 2

New instructions

- Action N means to move the waypoint north by the given value.
- Action S means to move the waypoint south by the given value.
- Action E means to move the waypoint east by the given value.
- Action W means to move the waypoint west by the given value.
- Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
- Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
- Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

In [34]:
class BetterShip:
    
    def __init__(self):
        self.y_coord = 0 # North / South co-ordinate
        self.x_coord = 0 # East / West co-ordinate
        self.waypoint_relative_x = 10
        self.waypoint_relative_y = 1
    
    
    def rotate_waypoint(self, rotation_direction, degrees):
        
        # Check validity
        if degrees not in [90, 180, 270]:
            raise ValueError('rotation degrees not multiple of 90')
        
        # Get quarters rotated by
        clockwise_quarter_rotations = int((360-degrees)/90 if rotation_direction == 'L' else degrees/90)
        
        old_relative_x = self.waypoint_relative_x
        old_relative_y = self.waypoint_relative_y
        
        # Set new relative waypoint co-ordinates based on rotation
        if clockwise_quarter_rotations == 1:
            # eg. from (10, 4) 1 quarter clockwise to (4, -10)
            self.waypoint_relative_x = old_relative_y
            self.waypoint_relative_y = -old_relative_x
        elif clockwise_quarter_rotations == 2:
            # eg. from (10, 4) 2 quarter clockwise to (-10, -4)
            self.waypoint_relative_x = -old_relative_x
            self.waypoint_relative_y = -old_relative_y
        elif clockwise_quarter_rotations == 3:
            # eg. from (10, 4) 3 quarter clockwise to (-4, 10)
            self.waypoint_relative_x = -old_relative_y
            self.waypoint_relative_y = old_relative_x
        else:
            print(rotation_direction, degrees, clockwise_quarter_rotations)
            raise ValueError('clockwise_quarter_rotations not between 1 and 3')
    
    
    def move_waypoint(self, compass_direction, spaces):
        
        if compass_direction not in ['N','E','S','W']:
            raise ValueError('Invalid compass_direction')
        elif compass_direction == 'N':
            self.waypoint_relative_y += spaces
        elif compass_direction == 'E':
            self.waypoint_relative_x += spaces
        elif compass_direction == 'S':
            self.waypoint_relative_y -= spaces
        else:
            self.waypoint_relative_x -= spaces
     
    
    def move_ship(self, spaces):
        
        self.x_coord += self.waypoint_relative_x * spaces
        self.y_coord += self.waypoint_relative_y * spaces
    
    
    def get_manhattan_distance(self):
        
        mh_dist = abs(self.x_coord) + abs(self.y_coord)
        print('x coord {}, y coord {}, MH distance {}'.format(self.x_coord,
                                                              self.y_coord,
                                                              mh_dist))

In [35]:
MyBetterShip = BetterShip()

for action, num in content:
    
    if action in ['R','L']:
        MyBetterShip.rotate_waypoint(action,num)
    elif action == 'F':
        MyBetterShip.move_ship(num)
    else:
        MyBetterShip.move_waypoint(action,num)

MyBetterShip.get_manhattan_distance()

x coord 91964, y coord -34833, MH distance 126797
